# ZapZap Awards!

### Bem vindo ao ZapZap Awards! Use as setinhas ← ↑ → ↓ pra navegar pelas paginas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from collections import defaultdict
import random
import requests
import json, string
import settings

%matplotlib inline
from IPython.display import Markdown, display
def printmd(s):
    display(Markdown(s))  
pd.set_option('display.max_colwidth', -1)

def display_nl(dataframe):
    from IPython.display import display, HTML
    return HTML(dataframe.to_html().replace("\\n","<br>") )


In [ ]:
STOP_WORDS = (requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-pt/master/stopwords-pt.json').json() + 
   'q pra hahaha to tá vc cara eh ta acho vou pq tô pro ja la'.split())

In [ ]:
df = pd.read_csv(f'{settings.GROUP_NAME}_messages.csv')
df.datetime = df.datetime.str.replace(' ', 'T')
df.loc[df.text.isnull(), 'text'] = ''

In [ ]:
df.head()

# Normalizing dataframe


In [ ]:
import re
import string
def sanitize(text):
    text = text.lower()
    text = re.sub(r'\bkkk+', 'kkkk', text)
    text = re.sub(r'\b(h[aeui]*){3,}', 'hahaha', text)
    return re.sub('[{}]'.format(string.punctuation), '', text)

def shortify_links(text):
    return re.sub(r'(https?://.*?/.{10})(.*?)( |$)', r'\1[...]', text)

print(sanitize('Todo dia, sièges. hahahahahaha haehaehaehea kkkkkkkk VAMO Q VAMO! 😀')) # TODO: enrich examples
df['stext'] = df.text.apply(sanitize)
df['short_links_text'] = df.text.apply(shortify_links)
df['datetime'] = pd.to_datetime(df.datetime)
df['date'] = df.datetime.dt.date

df.head()

# <center> Estatisticas por Pessoa </center>

Antes de comecarmos com os awards, vamos levantar umas estatisticas interessantes

## Horas favoritas

Aqui temos o numero de mensagens q cada um mandou ao longo do ano, ao longo das 24h de um dia

Tem gente q nao mandou uma unica mensagem as 5 da manha (no meu caso, foi shifted, para as 7)!

(Aperte ↓ pra entrar nesse topico) 

In [ ]:
msgs_per_hour = df.groupby(['name', lambda x: df.loc[x].datetime.hour ])


In [ ]:
people = defaultdict(lambda: {i:0 for i in range(24)})
PEOPLE = people
for name_hour, _df in msgs_per_hour:
    name, hour = name_hour
    count = _df.count().datetime
    people[name][hour] = count
for p in people:
    _df = pd.DataFrame(people[p], index=['count']).transpose()
    _df.plot.bar(legend=False, title=p).set(xlabel="hour of the day", ylabel="messages")

## Palavras caracteristicas

Quais as palavras que voce falou e que ninguem mais usa?

Omiti os nomes das pessoas nesse ae. Quero ver se vcs conseguem adivinhar quem eh quem (pelo menos os seus proprios).

In [ ]:
text_per_person = df.groupby('name').stext.apply(lambda x: ' '.join(x))
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.5)
tfidf_values = tfidf.fit_transform(text_per_person)

In [ ]:
_df = pd.DataFrame(tfidf_values.toarray(), index=text_per_person.index, columns=sorted(tfidf.vocabulary_))

In [ ]:
def word_cloud(dict_word_size):
    random.seed(46)
    from wordcloud import WordCloud
    dict_word_size = {k:5+v for k,v in dict_word_size.items() if v}
    wc = WordCloud(stopwords=[], color_func=lambda *_, **a: random.choice(['white','yellow','grey']), relative_scaling=1
                  , max_font_size=100, width=600, height=500, random_state=42 ).generate_from_frequencies(dict_word_size)
    plt.figure(figsize=(12, 10))
    plt.imshow(wc)
    plt.axis("off")
    plt.show()

In [ ]:
random.seed(46)
shuffled_people = list(PEOPLE)
random.shuffle(shuffled_people)
for idx, person in enumerate(shuffled_people):
    printmd(f'# Surfista #{person}')
    #printmd(f'# Surfista #{idx}')
    top_words = _df.loc[person].sort_values(ascending=False)[0:20]
    top_words.name = '--'
    print(top_words)
    if top_words[0] == 0: continue # skip people with no words
    word_cloud(top_words.to_dict())

# <center> Estatisticas do Surfe </center>

## Palavras favoritas

As palavras mais usadas no surfe, por mes.

Primeiro, filtradas por palavras [comuns](https://raw.githubusercontent.com/stopwords-iso/stopwords-pt/master/stopwords-pt.txt)

Depois, sem filtros

In [ ]:
from collections import Counter

for m in range(1,12+1): # TODO: change to month/year
    month_text = df[df.datetime.dt.month==(m)].stext.str.cat(sep='\n')
    words = [i for i in month_text.split() if i not in STOP_WORDS and len(i) > 1]
    if not words: continue
    printmd(f'# **Mes {m}**')
    counter = Counter(words)
    print(dict(counter.most_common(100)))
    word_cloud(dict(counter.most_common(100)))



In [ ]:
print('Agora sem nenhum filtro!')
month_text = df.stext.str.cat(sep='\n')
words = dict(Counter(month_text.split()).most_common(150))
print(words)
word_cloud(words)


Agora, sem mais delongas...


# Single Awards

Vamos a eles... os awards individuais! Ô familia, quarta feira abencoada...

## Troféu Metalinguistico - SAIU!
Quem falou mais as palavras 'surf' e 'night'
## Troféu Early riser - SAIU
Quem manda a primeira mensagem do dia (>6am)
## Troféu Wanderley - SAIU
Quem grita mais? Quem usou mais caps lock em 2017
## Troféu Machado de Assis
Quem tem o maior vocabulario? distinct words / words



## Troféu Pagerank
O mais citado
## Troféu Feliz
Quem ri mais?
## Troféu Fanfarrao
Quem mudou mais o subject do grupo
## Troféu Audiófilo
Quem mandou mais audio
## Troféu AlfiNET
Quem mandou mais imagens/videos
## Troféu Vacuo
Quem mandou a ultima mensagem da conversa (uma conversa) = 3h

# Segundo round

## Troféu NDP (sponsor Dibob)
Autoexplicativo
## Troféu Popstar
Quem foi mais chamado pelo nome próprio/apelidos
## Troféu Atention seeker
Mais chamou alguém por nome proprio/apelidos
## Troféu Amizade 
Quem mais xingou os amiguinhos
## Troféu Chulo (sponsor Chulé)
Quem mais falou palavrão
## Trofeu Mais Que Mil Palavras
Quem mais usou emoji

# Troféu Metalinguistico 🏆

Quem falou mais as palavras 'surf', 'night' e variantes

In [ ]:
_df = df[df.stext.str.contains('dota|cracu')][['short_links_text', 'name', 'datetime']]
pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_rows', -1)
display_nl(_df)

In [ ]:
_df.groupby('name').count().datetime.sort_values(ascending=False)

Com 19 citacoes, o ouro vai pro **BODE**! Prata pra mim e Bronze pro Chule! 

Eis ai um verdadeiro surfista noturno, um grande abracador da causa, sempre esta na atividade, estronda, farpacao... Um abraco mano debo! Comecando os trabalhos!

# Troféu Early riser 🏆
Quem mandou mais vezes a primeira mensagem do dia (>6am)


In [ ]:
msgs_per_day = df.groupby([lambda x: df.loc[x].datetime.date()])

In [ ]:
first_post_of_day = []
for day, _df in msgs_per_day:
    early = _df.loc[(_df.datetime.dt.hour > 6) & (_df.datetime.dt.hour < 10)]
    if not early.empty: first_post_of_day.append(early.sort_values(by='datetime').iloc[0].tolist())
first_post_of_day = pd.DataFrame(first_post_of_day, columns=early.columns)[['short_links_text', 'name', 'datetime']]
first_post_of_day

In [ ]:
first_post_of_day.groupby('name').count().datetime.sort_values(ascending=False)

Mas que momento! Temos claramente a familia caprina vencendo no quesito quem cedo madruga. Mano Jay vai levar uma medalha de bronze pra casa tambem!

Ceia, uma criatura naturalmente noturna, conseguiu empatar com o nosso trio ternura do sumico. Luke, David e o lendario Guilha.

Gostaria de enfatizar que ganhei do Law e do Kim. Q minha fama de dorminhoco seja revista!


## Troféu Wanderley 🏆
Quem grita mais? Quem usou mais caps lock em 2017

In [ ]:
text_per_name = df.groupby(['name']).text.apply(lambda x: ''.join(x)).to_dict()
letters_per_name = {n: Counter(t) for n, t in text_per_name.items()}
def get_small_big_ratio(counter):
    small, big = 0, 0
    acentos = 'áâãàçéêíóôõú'
    for l in string.ascii_lowercase + acentos: small += counter.get(l, 0)
    for l in string.ascii_uppercase + acentos.upper(): big += counter.get(l, 0)
    return 100*big/(small+big) if small + big else float('nan')
scores = { n: get_small_big_ratio(counter) for n, counter in letters_per_name.items()}
scores = sorted(scores.items(), key=lambda x: x[1])

In [ ]:
gritos = df.loc[np.logical_and(
            df.short_links_text.apply(lambda t:get_small_big_ratio(Counter(t))) > 51, df.text.str.len() > 5)
        ].copy()
grito_ratio = df.groupby('name').count().datetime/gritos.groupby('name').count().datetime


In [ ]:
gritos['order'] = gritos.name.apply(lambda n: grito_ratio[n] if not pd.isna(n) else float('inf'))
#display_nl(sort_values(by='name'))
display_nl(gritos.sort_values(['order', 'datetime'])['text name datetime'.split()])

In [ ]:
printmd('## Medido em mensagens normais por grito. \n ### Um grito eh qq mensagem com mais letras UPPERCASE do que lowercase. ' )
grito_ratio.sort_values()
#TODO: trocar pra gritos / 100 mensagens

Cara! Achei esse resultado mt curioso! 

Kimzola ganhou em primeiro lugar DISPARADO! 

Em segundo lugar o mano CEIA, quem diria cara. Ele fala pouco, mas fala gritando. 

E o Dibob, provavelmente o favorito, conseguiu chegar no top 3.

Parabens aos envolvidos!


# To be developed...
<img src="https://gcn.com/~/media/GIG/GCN/Redesign/Generic/AgileDevelopment.png">

In [ ]:
############################################################################################### APELIDAS #######

#display(df[df.stext.str.contains(r'\b(?:{})'.format(r'fre'), regex=True)])
df.stext.str.extract(r'.*\b({})'.format(r'fr.*?\b'), expand=False).dropna().unique()

In [ ]:
df.stext.str.contains('.*raf', regex=True)

In [ ]:
df[df.stext.str.match(r'.*\b({})'.format(r'.*o cao\b'))].head(100)

# Single Stats

## Favorite Hours
Normalizado por total de mensagens
## Palavras caracteristicas
As palavras que so vc fala


# Surf Stats

## Palavras favoritas

## Mensagens por hora/weekday


# Surf Awards

## Dia Mar Sinistro
Dia com maior numero de mensagens
## Combo
Dias consecutivos com mais de n mensagens
## AntiCombo
Dias consecutivos com 0 mensagens

# Surf Stats

## Palavras favoritas

## Mensagens por hora/weekday

In [ ]:
%%html
<style>
 .chat {
     display: flex;
     flex-direction: column;
 }
 .box-right {
     display: flex;
     justify-content: flex-end;
     
 }
 .box-left {
     display: flex;
     justify-content: flex-start;
 }
 .box-right-content {
    min-height: 30px;
    max-width: 40%;
    color: white;
    background: #ff974b;
    margin-right: 7px;
    padding: 10px;
    box-shadow: 1px 1px 3px 0px #CCC;
    border-radius: 10px;
    margin-bottom: 5px;
 }
 .box-left-content {
    min-height: 30px;
    max-width: 40%;
    background-color: #EDEDED;
    color: #4A4A4A;
    margin-right: 7px;
    padding: 10px;
    box-shadow: 1px 1px 3px 0px #CCC;
    border-radius: 10px;
    margin-bottom: 5px;
 }
 .chat-text {
    margin:0;
 }
 .chat-time {
    opacity: 0.6;
    font-size: 0.917em;
    line-height: 1.167em;
    margin:0 !important;
 }
 .box-info {     
     display: flex;
     justify-content: center;
 }
 .box-info-content {     
    background-color: #333;
    opacity: 0.5;
    padding: 5px;
    min-width: 125px;
    box-shadow: 1px 1px 3px 0px #CCC;
    border-radius: 10px;
    color: #FFF;
    text-align: center;
         font-size: 0.917em;


 }
</style>

In [ ]:
def owner_msg(text, time):
    return f'''<div class="box-right">
                 <div class="box-right-content"> <p class="chat-text">{text}</p> <p class="chat-time">{time}</p></div>
               </div>'''
def other_msg(author, text, time):
    return f'''<div class="box-left">
                 <div class="box-left-content"> <p class="chat-text">{author}: {text}</p> <p class="chat-time">{time}</p></div>
               </div>'''
def info_msg(text):
    return f'''<div class="box-info">
                     <div class="box-info-content"> {text} </div>
               </div>'''

In [ ]:
def display_chat(df, owner):
    html = ''
    html += info_msg('A very nice chat!')
    for idx, author, text, datetime in df[['name', 'text', 'datetime']][:50].to_records():
        html += other_msg(author, text, datetime) if author != owner else owner_msg(text, datetime)
    from IPython.display import display, HTML
    display(HTML(html))

display_chat(df, df.name[0])